In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-crdito/credito8.csv


**Esta análise realizada como projeto final do curso de *SQL para análise de dados* da EBAC.**
# O Objetivo: #

> Esse projeto foi buscou mostrar aplicações práticas de querys em SQL para realizar análises a respeito de uma base de dados com informações de clientes de um banco.

# Os Dados:
> As informações disponibilizadas em documento csv a respeito dos clientes foram:
* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses

A base está disponivel em: [dataset_crédito](http:///kaggle/input/dataset-crdito)

# A Metodologia:
> Para analisar as informações presentes nos dados, inicicalmente o documento .csv foi adicionado em um bucket no sistema de armazenamento S3 e em seguida analisado usando o sistema de processamento Athena, ambas ferramentas disponiveis no pacote AWS.

> Em geral, inicialmente foi feita uma análise exploratória para compreensão dos dados disponíveis, seguida de uma análise mais crítica dos dados comparando a prevalência de dados e comparando fatores relevantes entre si. Por fim, as conclusões a respeito da base de dados foram elencadas.


## Análise exploratória:
Em primeiro lugar, foi gerada uma prévia com 10 linhas dos dados alimentados no S3:
>![tabela](https://github.com/gabrielsosil/sqlebaca/blob/main/Captura%20de%20tela%202023-04-21%20142925.png?raw=true)
**query:** SELECT * FROM credito limit 10;

Para um melhor entendimento dos tipos de dados e quais análises são possiveis de serem feitas, se buscou qual o tipo de dado em cada coluna:

>![descricao](https://github.com/gabrielsosil/sqlebaca/blob/main/datatype.png?raw=true)

**query:** DESCRIBE credito

Além da quantidade de clientes que seriam analisados:
>![quant](https://github.com/gabrielsosil/sqlebaca/blob/main/dataquant.png?raw=true)

**query:** SELECT COUNT(* ) FROM credito

Para uma análise mais granular, foi feita uma análise dos dados tipo *string* para entender quais as ocorrências em cada coluna:
>![escolaridade](https://github.com/gabrielsosil/sqlebaca/blob/main/escolaridade.png?raw=true/)
>![civil](https://github.com/gabrielsosil/sqlebaca/blob/main/estadocivil.png?raw=true)
>![salario](https://github.com/gabrielsosil/sqlebaca/blob/main/salario_anua.png?raw=true)
>![tipocart](https://github.com/gabrielsosil/sqlebaca/blob/main/tipo_cartao.png?raw=true)

>Para as informações acima, foram usadas **querys**: SELECT DISTINCT (coluna) FROM credito , substituindo a coluna pelo dado desejado.

>Nota-se também a presença de *'na'* em diferentes colunas. Estas são devidas a preenchimento inadequado ou não realizado. Tais dados deverão ser tratados em análises mais complexas.


## Análise dos dados

> **Uma boa maneira de começar** a entender os dados é buscar a frequência das categorias nas diferentes colunas. Nesse caso é possivel aprender sobre o perfil dos clientes contemplados. Esses dados são essenciais para direcionar campanhas de marketing e oferecer planos especiais, por exemplo. Seguem exemplos desse tipo de análise:

![sexo](https://github.com/gabrielsosil/sqlebaca/blob/main/sexo.png?raw=true)
![faixa](https://github.com/gabrielsosil/sqlebaca/blob/main/faixa%20salarial.png?raw=true)
![Escolaridade](https://github.com/gabrielsosil/sqlebaca/blob/main/escolaridade.png?raw=true)

>Para as informações acima, foram usadas **querys**: SELECT COUNT (*), (categoria) FROM credito GROUP BY (gategoria) , substituindo 'gategoria' pelo dado desejado.


> **Além da frequência de dados,** também é possivel selecionar dados baseado na média de categorias diferentes, para estipular o comportamento médio frente a alguma situação:

![sexogasto](https://github.com/gabrielsosil/sqlebaca/blob/main/gastosexo.png?raw=true)
![tipocartaogasto](https://github.com/gabrielsosil/sqlebaca/blob/main/tipocartaogasto.png?raw=true)

>Para as informações acima, foram usadas **querys**: SELCT AVG(valor_transacoes_12m) AS media_valor_gasto, (gategoria) FROM credito GROUP BY (categoria) , substituindo 'categoria' pelo dado buscado.

> **Também é possível, partindo de uma caracteristica,** determinar outros fatores que estão correlacionados à ela, como por exemplo para determinar o perfil dos clientes com os maiores e menores limites:

>![limitealto](https://github.com/gabrielsosil/sqlebaca/blob/main/perfillimitealto.png?raw=true)

>**query**: select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito desc limit 10 

>![limitebaixo](https://github.com/gabrielsosil/sqlebaca/blob/main/limitebaixo.png?raw=true)

>**query**: select max(limite_credito) as limite_credito, escolaridade, tipo_cartao, sexo from credito where escolaridade != 'na' and tipo_cartao != 'na' group by escolaridade, tipo_cartao, sexo order by limite_credito asc limit 10 

> Por fim, as análises em si também podem gerar mais perguntas, e permitem a volta para criação de mais querys:

![limitexsexo](https://github.com/gabrielsosil/sqlebaca/blob/main/limitexsexo.png?raw=true)
> **query**:SELECT AVG(limite_credito) AS media_limite, sexo FROM credito GROUP BY sexo permitiu a análise da média dos limites dos cartões separados por sexo.

# Conclusões: #

* A maior parte dos clientes é do sexo masculino, possui renda até 40K e nivel de educação até o Mestrado.
* Os sexo não parece influenciar no gasto.
* Apesar disso, os clientes masculinos tem maiores limites e os clientes femininos tem menores limites.

**Uma análise mais completa e granular dos dados pode ter a capacidade de explicar essa diferença de limites, assim como a constatação dessas diferenças pode ser um motivo para as empresas perceberem a necessidade de mudança no tratamento das clientes femininas!**

